In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import lasio
from scipy.optimize import curve_fit
from math import *

%matplotlib qt

# Загрузка данных ГИС

In [2]:
current_file_dir = os.path.abspath('')
data_dir = os.path.join(current_file_dir, 'data')
for file in os.listdir(data_dir):
    if file.endswith('.las'):
        data_file_dir = os.path.join(data_dir, file)
        las = lasio.read(data_file_dir, ignore_header_errors=True)
        df = las.df().reset_index()

# Отображение датафрейма с исходными данными ГИС

In [3]:
df

,MD,AZI,BS,CALI,DCAL,DEV,DRHO,DTC,DTS,GR,NPHI,PEF,RDEP,RHOB,RMED
0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.1,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.2,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.3,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.4,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39027,3902.7,0.0,8.5,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.970829,NaN,3.087276
39028,3902.8,0.0,8.5,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.960968,NaN,3.078955
39029,3902.9,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39030,3903.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Построение планшета с ГИС

In [4]:
# построение профилей исходных данных ГИС
fig = plt.subplots(figsize=(10, 10))
# ax1 - график для DTC/DTP (кросс-дипольный акустический каротаж)
ax1 = plt.subplot2grid((1, 3), (0, 0), rowspan=1, colspan=1)
# ax2 - график для DTS (широкополостный акустический каротаж); строится там же, что и ax1
ax2 = ax1.twiny()
# ax3 - график для RHOB (гамма-гамма плотностной каротаж)
ax3 = plt.subplot2grid((1, 3), (0, 1), rowspan=1, colspan=1, sharey=ax1)

ax1_color = 'blue'
ax2_color = 'magenta'
ax3_color = 'red'

# DTC
# построение графика
ax1.plot(df['DTC'], df['MD'], color=ax1_color, lw=0.5)
# установка границ диапазона по оси абсцисс
ax1.set_xlim(720, 120)
# установка границ диапазона по оси ординат
ax1.set_ylim(4000, 0)
# создание подписей к осям
ax1.set_xlabel('DTC, μs/m')
ax1.set_ylabel('MD, m')
# присвоение цвета подписи
ax1.xaxis.label.set_color(ax1_color)
# расположение подписи к обозначению линии
ax1.spines['top'].set_position(('axes', 1.00))
# присвоение цвета отметкам основных линий сетки
ax1.tick_params(axis='x', colors=ax1_color)
# присвоение цвета оси
ax1.spines['top'].set_edgecolor(ax1_color)
# создание сетки
ax1.grid(which='both', color='lightgrey', linestyle='-')
# расположение главных отметок и подписи (сверху)
ax1.xaxis.set_ticks_position('top')
ax1.xaxis.set_label_position('top')

# DTS
ax2.plot(df['DTS'], df['MD'], color=ax2_color, lw=0.5)
ax2.set_xlim(1200, 200)
ax2.set_ylim(4000, 0)
ax2.set_xlabel('DTS, μs/m')
ax2.set_ylabel('MD, m')
ax2.xaxis.label.set_color(ax2_color)
ax2.spines['top'].set_position(('axes', 1.08))
ax2.tick_params(axis='x', colors=ax2_color)
ax2.spines['top'].set_edgecolor(ax2_color)
ax2.xaxis.set_ticks_position('top')
ax2.xaxis.set_label_position('top')

# RHOB
ax3.plot(df['RHOB'], df['MD'], color=ax3_color, lw=0.5)
ax3.set_xlim(2, 3)
ax3.set_ylim(4000, 0)
ax3.set_xlabel('RHOB, g/cm3')
ax3.xaxis.label.set_color(ax3_color)
ax3.spines['top'].set_position(('axes', 1.00))
ax3.tick_params(axis='x', colors=ax3_color)
ax3.spines['top'].set_edgecolor(ax3_color)
ax3.grid(which='both', color='lightgrey', linestyle='-')
ax3.minorticks_on()
ax3.xaxis.set_ticks_position('top')
ax3.xaxis.set_label_position('top')

plt.setp(ax3.get_yticklabels(), visible=False)
plt.show()

C:\Users\almuravtsev\AppData\Local\Temp\ipykernel_7852\1820084643.py:4: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  ax1 = plt.subplot2grid((1, 3), (0, 0), rowspan=1, colspan=1)


# Функция расчёта TVD и отходов от вертикали (методом минимальной кривизны)

In [5]:
def tvd_calculation(
    df: pd.DataFrame
) -> pd.DataFrame:
    df['TVD'] = nan
    df['EAST'] = nan
    df['NORTH'] = nan
    df['DISPLACEMENT'] = nan
    
    df.at[0, 'TVD'] = 0
    df.at[0, 'EAST'] = 0
    df.at[0, 'NORTH'] = 0
    df.at[0, 'DISPLACEMENT'] = 0
    
    if 'DEV' in df.columns and 'AZI' in df.columns:
        for i, row in df.iloc[1:].iterrows():
            i1 = radians(df.at[i-1, 'DEV'])
            i2 = radians(df.at[i, 'DEV'])
            a1 = radians(df.at[i-1, 'AZI'])
            a2 = radians(df.at[i, 'AZI'])
            beta = radians(acos(cos(i2 - i1) - sin(i1) * sin(i2) * (1 - cos(a2 - a1))))
            if beta == 0:
                df.at[i, 'TVD'] = df.at[i-1, 'TVD'] + (df.at[i, 'MD'] - df.at[i-1, 'MD']) * cos(i2)
                df.at[i, 'EAST'] = df.at[i-1, 'EAST'] + (df.at[i, 'MD'] - df.at[i-1, 'MD']) * sin(i2) * sin(a2)
                df.at[i, 'NORTH'] = df.at[i-1, 'NORTH'] + (df.at[i, 'MD'] - df.at[i-1, 'MD']) * sin(i2) * cos(a2)
            else:
                rf = 2 / beta * tan(beta / 2)
                df.at[i, 'TVD'] = df.at[i-1, 'TVD'] + (df.at[i, 'MD'] - df.at[i-1, 'MD']) / 2 * (cos(i1) + cos(i2)) * rf
                df.at[i, 'EAST'] = df.at[i-1, 'EAST'] + (df.at[i, 'MD'] - df.at[i-1, 'MD']) / 2 * \
                    (sin(i1) * sin(a1) + sin(i2) * sin(a2)) * rf
                df.at[i, 'NORTH'] = df.at[i-1, 'NORTH'] + (df.at[i, 'MD'] - df.at[i-1, 'MD']) / 2 * \
                    (sin(i1) * cos(a1) + sin(i2) * cos(a2)) * rf
            df.at[i, 'DISPLACEMENT'] = sqrt((df.at[i, 'NORTH']) ** 2 + (df.at[i, 'EAST']) ** 2)
    else:
        for i, row in df.iterrows():
            df.at[i, 'TVD'] = df.at[i, 'MD']
            df.at[i, 'DISPLACEMENT'] = sqrt((df.at[i, 'NORTH']) ** 2 + (df.at[i, 'EAST']) ** 2)
    
    return df

# Расчёт TVD и отходов от вертикали

In [6]:
df = tvd_calculation(df)

# Построение проекций и 3D-графика траектории

In [7]:
#построение графиков траектории
fig = plt.figure()
#подпись графиков
plt.title('34_5-1S', fontsize = 20)
#отключение рамки вокруг области построения
plt.axis("off")
#задание предельных значений для осей
#limit_plan = df['DISPLACEMENT'].max()+200
limit_plan = 2000
limit_tvd = df['TVD'].max() + 200

#построение профиля траектории в координатах "отход от вертикали - TVD"
ax = fig.add_subplot(1, 3, 1)
ax.set_xlim([-limit_plan, limit_plan])
ax.set_ylim([-100, limit_tvd])
ax.grid(True)
ax.invert_yaxis()
ax.plot(df['DISPLACEMENT'], df['TVD'])
#задание координаты устья
ax.scatter(df['DISPLACEMENT'].iloc[0], df['TVD'].iloc[0])
ax.set_aspect('equal', adjustable='box')
ax.annotate('Устье',
            xy=(df['DISPLACEMENT'].iloc[0], df['TVD'].iloc[0]), xycoords='data',
            xytext=(-30, 5), textcoords='offset points')
#задание координаты забоя
ax.scatter(df['DISPLACEMENT'].iloc[-1], df['TVD'].iloc[-1], marker='*')
ax.annotate('Забой',
            xy=(df['DISPLACEMENT'].iloc[-1], df['TVD'].iloc[-1]),
            xycoords='data', xytext=(-40, -10), textcoords='offset points')
ax.set_xlabel("Отход от вертикали, м")
ax.set_ylabel('TVD, м')
ax.set_aspect(1)
ax.set_title('Профиль', size = 20)

#построение плана траектории в координатах "Отклонение на восток - отклонение на север"
ax = fig.add_subplot(1,3,2)
ax.set_xlim([-limit_plan, limit_plan])
ax.set_ylim([-limit_plan, limit_plan])
ax.set_aspect('equal', adjustable='box')
ax.plot(df['EAST'], df['NORTH'])
ax.grid(True)
#задание координаты устья
ax.scatter(df['EAST'].iloc[0], df['NORTH'].iloc[0])
ax.annotate('Устье',
            xy=(df['EAST'].iloc[0], df['NORTH'].iloc[0]), xycoords='data',
            xytext=(-30, 10), textcoords='offset points')
#задание координаты забоя
ax.scatter(df['EAST'].iloc[-1], df['NORTH'].iloc[-1], marker='*')
ax.annotate('Забой',
            xy=(df['EAST'].iloc[-1], df['NORTH'].iloc[-1]),
            xycoords='data', xytext=(-40, -10), textcoords='offset points')
ax.set_xlabel('Отклонение на восток, м')
ax.set_ylabel('Отклонение на север, м')
ax.set_title('План', size = 20)

#построение трёхмерного графика
ax = fig.add_subplot(1, 3, 3, projection='3d')
ax.set_ylim(-limit_plan, limit_plan)
ax.set_xlim(-limit_plan, limit_plan)
ax.set_zlim(-100, limit_tvd)
ax.plot(df['EAST'], df['NORTH'], df['TVD'])
ax.set_aspect('auto', adjustable='box')
ax.invert_zaxis()
ax.set_xlabel('Отклонение на восток, м')
ax.set_ylabel('Отклонение на север, м')
ax.set_zlabel('TVD, м')
ax.view_init(elev=25, azim=240)
ax.set_title('Пространственная траектория', size = 20)

plt.show()

# Функция расчёта вертикального напряжения

In [8]:
#расчёт вертикального напряжения
def overburden(
    df: pd.DataFrame
) -> tuple:
    #задание глобальных переменных, которые инициализируются
    #за пределами тела функции. Необходимо для того,
    #чтобы не добавлять большого количества аргументов в функцию.
    global surface
    global air_gap
    global a_tvd
    global b_tvd
    global a_rho
    global b_rho
    
    df['Overburden'] = nan
    df['Pore Pressure'] = nan
    df['RHOB_extrapolated'] = nan
 
    #задание опорных точек для интерполяции плотности
    points = {air_gap: surface,
               a_tvd: a_rho,
               b_tvd: b_rho}
    #создание списков из значений TVD и RHOB
    xdata = list(points.keys())
    ydata = list(points.values())
    
    #функция для экстраполяции плотности
    def func(x, a, b):
        rho = surface + a * (x - air_gap) ** b
        return rho
    
    #поиск коэффициентов для экстраполяции плотности
    popt, _ = curve_fit(func, xdata, ydata, maxfev=10000)
    a, b = popt
    
    #Задание начальных значений при TVD=0
    df.at[0, 'Overburden'] = 0
    df.at[0, 'RHOB_extrapolated'] = 0
    
    #расчёт экстраполированной плотносnи, вертикального напряжения и пластового давления
    for i, row in df.iloc[1:].iterrows():
        if isnan(df.at[i,'RHOB']):
            if df.at[i, 'TVD'] <= air_gap:
                df.at[i, 'RHOB_extrapolated'] = 0
            else:    
                df.at[i,'RHOB_extrapolated'] = func(df.at[i, 'TVD'], a, b)
        else:
            df.at[i,'RHOB_extrapolated'] = df.at[i, 'RHOB']
        
        tvd_1 = df.at[i-1, 'TVD']
        tvd_2 = df.at[i, 'TVD'] 
        rhob = df.at[i, 'RHOB_extrapolated']
        
        df.at[i, 'Overburden'] = df.at[i-1, 'Overburden'] + rhob * (tvd_2 - tvd_1) * 9.81 / 1000

    return df['Overburden'], df['RHOB_extrapolated']

# Функция расчёта пластового давления

In [9]:
# расчёт гидростатического давления
def hydrostatic(
    df: pd.DataFrame
):
    # задание глобальных переменных, которые инициализируются
    # за пределами тела функции. Необходимо для того,
    # чтобы не добавлять большого количества аргументов в функцию.
    global air_gap
    global hydrostatic_gradient
    
    df['Hydrostatic'] = nan
    df['Hydrostatic Gradient'] = nan

    df.at[0, 'Hydrostatic'] = 0

    # расчёт экстраполированной плотносnи и вертикального напряжения
    for i, row in df.iloc[1:].iterrows():
        if df.at[i, 'TVD'] <= air_gap:
            df.at[i, 'Hydrostatic Gradient'] = 0
        else:    
            df.at[i,'Hydrostatic Gradient'] = hydrostatic_gradient  # кПа/м
        
        tvd_1 = df.at[i-1, 'TVD']
        tvd_2 = df.at[i, 'TVD'] 
        
        df.at[i,'Hydrostatic'] = df.at[i-1,'Hydrostatic'] + df.at[i,'Hydrostatic Gradient'] * (tvd_2 - tvd_1) / 1000

    return df['Hydrostatic'], df['Hydrostatic Gradient']


def shale_flag(
    df: pd.DataFrame
):
    
    global gr_sand
    global gr_shale
    
    df['VSH'] = nan
    
    # расчёт флага глинистости
    for i, row in df.iloc[1:].iterrows():
        if isnan(df.at[i, 'GR']):
            df.at[i, 'VSH'] = 0
        else:    
            df.at[i,'VSH'] = (df.at[i,'GR'] - gr_sand) / (gr_shale - gr_sand)
        
        if  df.at[i,'VSH'] < 0:
            df.at[i,'VSH'] = 0
        elif df.at[i,'VSH'] > 1:
            df.at[i,'VSH'] = 1
            
    return df['VSH']


def dtc_trend(
    df: pd.DataFrame
):
    # задание глобальных переменных, которые инициализируются
    # за пределами тела функции. Необходимо для того,
    # чтобы не добавлять большого количества аргументов в функцию.
    global surface
    global air_gap
    global a_dtc_tvd
    global b_dtc_tvd
    global a_dtc
    global b_dtc
    
    df['DTC Trend'] = nan
 
    # задание опорных точек для интерполяции плотности
    points = {a_dtc_tvd: a_dtc,
               b_dtc_tvd: b_dtc}
    
    # функция для экстраполяции DTC
    def func(x):
        k = (b_dtc - a_dtc) / (b_dtc_tvd - a_dtc_tvd)
        b = b_dtc - k * b_dtc_tvd
        dtc = k * x + b
        # dtc = a_dtc_tvd * 10 ** ((x - a_dtc_tvd) / (b_dtc_tvd - a_dtc_tvd) * log10(b_dtc / a_dtc))
        
        return dtc
    
    # Задание начальных значений при TVD=0
    df.at[0, 'Overburden'] = 0
    df.at[0, 'RHOB_extrapolated'] = 0
    
    # расчёт экстраполированной плотносnи, вертикального напряжения и пластового давления
    for i, row in df.iloc[1:].iterrows():
        if df.at[i, 'TVD'] > 2600:
            df.at[i, 'DTC Trend'] = func(df.at[i, 'TVD'] )
        else:
            df.at[i, 'DTC Trend'] = nan

    return df['DTC Trend']


# расчёт по Итону
def eaton(
    df: pd.DataFrame
):
    # задание глобальных переменных, которые инициализируются
    # за пределами тела функции. Необходимо для того,
    # чтобы не добавлять большого количества аргументов в функцию.
    global air_gap
    
    df['Eaton'] = nan
    df['Eaton Gradient'] = nan

    df.at[0, 'Eaton'] = 0

    # расчёт гидростатического давления
    for i, row in df.iloc[1:].iterrows():
        if isnan(df.at[i, 'DTC']):
            df.at[i, 'Eaton Gradient'] = df.at[i, 'Hydrostatic Gradient']
        else: 
            if df.at[i, 'DTC Trend'] < df.at[i, 'DTC'] and df.at[i, 'VSH'] >= 0.4:
                df.at[i, 'Eaton Gradient'] = (df.at[i, 'RHOB_extrapolated'] * 9.81 - \
                    (df.at[i, 'RHOB_extrapolated'] * 9.81 - df.at[i, 'Hydrostatic Gradient']) * \
                        (df.at[i, 'DTC Trend'] / df.at[i, 'DTC']) ** 3)
            else:
                df.at[i,'Eaton Gradient'] = nan
                
    df['Eaton Gradient'].interpolate(method='linear', inplace=True)
    for i, row in df.iloc[1:].iterrows():
        tvd_1 = df.at[i-1, 'TVD']
        tvd_2 = df.at[i, 'TVD'] 
        
        # df.at[i, 'Eaton'] = df.at[i-1, 'Eaton']+df.at[i, 'Eaton Gradient'] * (tvd_2 - tvd_1) / 1000
        df.at[i, 'Eaton'] = df.at[i, 'Eaton Gradient'] * df.at[i, 'TVD'] / 1000
        
    return df['Eaton'], df['Eaton Gradient']

# Задание значений глобальных переменных и расчёт вертикального напряжения и пластового давления

In [10]:
# Задание глобальных переменных для расчёта
surface = 1.6
air_gap = 6
a_tvd = 2151
b_tvd = 3847
a_rho = 2.27
b_rho = 2.5

hydrostatic_gradient = 10 #кПа/м

gr_sand = 90
gr_shale = 110

a_dtc_tvd = 2808
b_dtc_tvd = 3835
a_dtc = 365
b_dtc = 280


df['Overburden'], df['RHOB_extrapolated'] = overburden(df)   
df['Hydrostatic'], df['Hydrostatic Gradient'] = hydrostatic(df)
df['VSH'] = shale_flag(df)
df['DTC Trend'] = dtc_trend(df)
df['Eaton'], df['Eaton Gradient'] = eaton(df)

# Построение графиков результатов расчёта

In [11]:
#Построение профилей вертикального напряжения, пластового давления и объёмной плотности
fig, axes = plt.subplots()
#ax1.set_ylabel('MD, м')
        
ax1 = plt.subplot2grid((1,4), (0,0), rowspan=1, colspan=1)
ax2 = ax1.twiny()
ax3 = ax1.twiny()
ax4 = plt.subplot2grid((1,4), (0,1), rowspan=1, colspan=1, sharey = ax1)
ax5 = ax4.twiny()
ax6 = plt.subplot2grid((1,4), (0,2), rowspan=1, colspan=1, sharey = ax1)
ax7 = plt.subplot2grid((1,4), (0,3), rowspan=1, colspan=1, sharey = ax1)
ax8 = ax7.twiny()


#Вертикальное напряжение
ax1.plot(df['Overburden'], df['MD'], color='black', lw=1)
ax1.set_xlim(0, 100)
ax1.set_xlabel('Sv, МПа')
ax1.xaxis.label.set_color('black')
ax1.spines['top'].set_position(('axes', 1.00))
ax1.tick_params(axis='x', colors='black')
ax1.spines['top'].set_edgecolor('black')
ax1.set_ylim(4000, 0)
ax1.grid(which='both', color='lightgrey', linestyle='-')
ax1.xaxis.set_ticks_position('top')
ax1.xaxis.set_label_position('top')
ax1.set_title('34_5-1S')
ax1.set(ylabel='MD, м')
ax1.label_outer()


#Пластовое давление
ax2.plot(df['Eaton'], df['MD'], color='c', lw=1)
ax2.set_xlim(0, 100)
ax2.set_xlabel('Пластовое давление (Итон), МПа')
ax2.xaxis.label.set_color('c')
ax2.spines['top'].set_position(('axes', 1.05))
ax2.tick_params(axis='x', colors='c')
ax2.spines['top'].set_edgecolor('c')
ax2.minorticks_on()
ax2.xaxis.set_ticks_position('top')
ax2.xaxis.set_label_position('top')
ax2.set_ylim(4000, 0)

#Пластовое давление
ax3.plot(df['Hydrostatic'], df['MD'], color='magenta', lw=1)
ax3.set_xlim(0, 100)
ax3.set_xlabel('Пластовое давление (Гидростатика), МПа')
ax3.xaxis.label.set_color('magenta')
ax3.spines['top'].set_position(('axes', 1.1))
ax3.tick_params(axis='x', colors='magenta')
ax3.spines['top'].set_edgecolor('magenta')
ax3.minorticks_on()
ax3.xaxis.set_ticks_position('top')
ax3.xaxis.set_label_position('top')
ax3.set_ylim(4000, 0)

#Градиенты
ax4.plot(df['Eaton Gradient'], df['MD'], color='c', lw=0.5)
ax4.set_xlim(0, 20)
ax4.set_xlabel('Градиент (Итон)')
ax4.xaxis.label.set_color('c')
ax4.spines['top'].set_position(('axes', 1.00))
ax4.tick_params(axis='x', colors='c')
ax4.spines['top'].set_edgecolor('c')
ax4.grid(which='both', color='lightgrey', linestyle='-')
ax4.minorticks_on()
ax4.xaxis.set_ticks_position('top')
ax4.xaxis.set_label_position('top')
ax4.set_ylim(4000, 0)
plt.setp(ax4.get_yticklabels(), visible=False)

ax5.plot(df['Hydrostatic Gradient'], df['MD'], color='magenta', lw=0.5)
ax5.set_xlim(0, 20)
ax5.set_xlabel('Градиент (Гидростатика)')
ax5.xaxis.label.set_color('magenta')
ax5.spines['top'].set_position(('axes', 1.05))
ax5.tick_params(axis='x', colors='magenta')
ax5.spines['top'].set_edgecolor('magenta')
ax5.grid(which='both', color='lightgrey', linestyle='-')
ax5.minorticks_on()
ax5.xaxis.set_ticks_position('top')
ax5.xaxis.set_label_position('top')
ax5.set_ylim(4000, 0)
plt.setp(ax5.get_yticklabels(), visible=False)

ax6.plot(df['VSH'], df['MD'], color='red', lw=0.5)
ax6.set_xlim(0, 1)
ax6.set_xlabel('Глинистость')
ax6.xaxis.label.set_color('red')
ax6.spines['top'].set_position(('axes', 1.00))
ax6.tick_params(axis='x', colors='red')
ax6.spines['top'].set_edgecolor('red')
ax6.grid(which='both', color='lightgrey', linestyle='-')
ax6.minorticks_on()
ax6.xaxis.set_ticks_position('top')
ax6.xaxis.set_label_position('top')
ax6.set_ylim(4000, 0)
plt.setp(ax6.get_yticklabels(), visible=False)


ax7.plot(df['DTC'], df['MD'], color='blue', lw=0.5)
ax7.set_xlim(720, 120)
ax7.set_xlabel('DTP, µs/m')
ax7.set_xscale('log')
ax7.set_ylabel('MD, m')
ax7.xaxis.label.set_color('blue')
ax7.spines['top'].set_position(('axes', 1.00))
ax7.tick_params(axis='x', colors='blue')
ax7.spines['top'].set_edgecolor('blue')
ax7.grid(which='both', color='lightgrey', linestyle='-')
ax7.xaxis.set_ticks_position('top')
ax7.xaxis.set_label_position('top')
ax7.set_ylim(4000, 0)
plt.setp(ax7.get_yticklabels(), visible=False)


ax8.plot(df['DTC Trend'], df['MD'], color='blue', lw=0.5)
ax8.set_xlim(720, 120)
ax8.set_xlabel('DTP, µs/m')
ax8.set_ylabel('MD, m')
ax8.set_xscale('log')
ax8.xaxis.label.set_color('blue')
ax8.spines['top'].set_position(('axes', 1.05))
ax8.tick_params(axis='x', colors='blue')
ax8.spines['top'].set_edgecolor('blue')
ax8.grid(which='both', color='lightgrey', linestyle='-')
ax8.xaxis.set_ticks_position('top')
ax8.xaxis.set_label_position('top')
ax8.set_ylim(4000, 0)
plt.setp(ax8.get_yticklabels(), visible=False)

plt.show()

C:\Users\almuravtsev\AppData\Local\Temp\ipykernel_7852\1543972185.py:5: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  ax1 = plt.subplot2grid((1,4), (0,0), rowspan=1, colspan=1)
